# Teleconnect Data Pipeline

End-to-end telecom data pipeline:
- extraction
- quality checks
- cleaning
- validation
- feature engineering
- loading


In [1]:
#import pandas as pd
#import numpy as np
from sqlalchemy import create_engine

In [2]:
import sys
sys.executable


'c:\\Users\\kay2c\\Downloads\\TeleconnectPipeline\\.venv\\Scripts\\python.exe'